<a href="https://colab.research.google.com/github/shaikadish/imdbProject/blob/main/rating_predictor_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to my Review Rating Predictor demo!

Before we run the demo, we need to load in important files. Hover the mouse over the code block directly bellow, and press the play button in the top left. This can take a minute or two. Scroll down to the **Demo** heading to run the demo.


In [5]:
# Remove all printed output. I don't want to scare any non-technical people!
%%capture

# Download model and library
!pip install transformers
!gdown --id 1-00HNjkWgzq6_b-UDJtUfztR5Ga-Ymbi

# Disable output watnings
import warnings
warnings.filterwarnings("ignore")

# Import libraties
import torch
import torch.nn as nn
from transformers import BertTokenizer, AutoModel
import transformers
transformers.logging.set_verbosity_error()

# Model class
class RatingModel(nn.Module):
    def __init__(self):
        super(RatingModel, self).__init__()
        
        # Load pretrained BERT model
        self.base_model = AutoModel.from_pretrained('fabriceyhc/bert-base-uncased-imdb')
        # Include dropout layer in classification head
        self.dropout = nn.Dropout(0.5)
        # Add new output layer for classification
        self.linear = nn.Linear(768, 1) 
        
    def forward(self, tokenized_reviews, attn_mask):

        # Pass the tokenized reviews through the BERT model
        outputs = self.base_model(tokenized_reviews, attention_mask=attn_mask)
        outputs = self.dropout(outputs[0])

        # The slicing of the BERT model outputs (bellow) is to make use of the BERT features 
        # from the only the special [CLS] token for classification. This token is at index 0
        outputs = self.linear(outputs[:,0,:])
        
        return outputs

# Predictor class
class RatingPredictor():
    def __init__(self,model,tokenizer):

      # Load model and tokenizer
      self.model = model.eval()
      self.tokenizer = tokenizer

    def predict(self,review):

      # Tokenize review
      encoded_dict = self.tokenizer.encode_plus(
                        review,                      
                        truncation=True,
                        add_special_tokens = True, 
                        max_length = 512,          
                        pad_to_max_length = True,
                        return_attention_mask = True,
                                           )
          
      tokenized_review=encoded_dict['input_ids']
      attention_mask=encoded_dict['attention_mask']

      # Get rating prediction
      preds=self.model(torch.tensor([tokenized_review]),torch.tensor([attention_mask]))

      # Return rating out of 10
      return round(preds.item()*10)

# Download tokenizer
tokenizer = BertTokenizer.from_pretrained('fabriceyhc/bert-base-uncased-imdb', do_lower_case=True)

# Create model and load
model = RatingModel()
model.load_state_dict(torch.load('imdb_1.pt',map_location=torch.device('cpu')))

# Create predictor
predictor = RatingPredictor(model,tokenizer)

# Demo

To run the demo, hover the mouse over the code block bellow, and press the play button. You can then type your review into the box that appears, and press enter to see the predicted rating. You can run this box as many times as you want! 

Remember, the model performs best if you type longer reviews, so if it is giving nonsensical ratings from reviews like "That movie sucked" try fleshing out the review to get a more accurate rating.

In [ ]:
review=input("Type a review: \n")
prediction=predictor.predict(review)
print(f'Predicted rating: {prediction}/10')

If you are interested in how I made this, please check out this [tutorial](https://github.com/shaikadish/imdbProject) I wrote which details the entire process!